# Description of variational inference approach to estimating a DPGMM

We assume that the data follows a Dirichlet process Gaussian mixture model:

$$
\begin{aligned}
\theta_k | \alpha_\circ & \sim \text{Beta}(1,\alpha_\circ) \equiv p(\theta) & [\forall k=1,\ldots,\infty]\\
\mu_k | \nu_\circ, \sigma_\circ & \sim \text{Normal}(\nu_\circ, \sigma^2_\circ) \equiv p(\mu) & [\forall k=1,\ldots,\infty] \\
\tau_k | \beta_\circ, \gamma_\circ & \sim \text{Gamma}(\beta_\circ, \gamma_\circ) \equiv p(\tau) & [\forall k=1,\ldots,\infty] \\
\pi_k | \theta_1,\ldots,\theta_k & = \theta_k\prod_{\ell=1}^{k-1}(1-\theta_\ell) & [\forall k=1,\ldots,\infty]\\
k_n | \pi_1,\ldots,\pi_\infty & \sim \text{Categorical}(\pi_1,\ldots,\pi_\infty) & [\forall n=1,\ldots,N]\\
x_n | k_n, \mu_{k_n}, \tau_{k_n} & \sim \text{Normal}(\mu_{k_n}, \tau^{-1}_{k_n}) & [\forall n=1,\ldots,N]
\end{aligned}
$$

The data generating process above makes use of the stick-breaking representation of the Dirichlet process. Given hyperpriors $(\alpha_\circ, \nu_\circ, \sigma_\circ, \beta_\circ, \gamma_\circ)$ and data $\mathcal{D}\equiv\{x_n\}_{n=1,\ldots,N}$, our goal is to infer the posterior distribution over the model. This repository presents a variational inference (VI) approach towards this end.

A reference worth pointing out is [Blei and Jordan (2006)](https://projecteuclid.org/download/pdf_1/euclid.ba/1340371077), which introduces a truncated generalized stick-breaking process as a variational posterior for the Dirichlet process prior. We follow this approach but, importantly, we make use of the reparameterization trick to estimate the gradient of the evidence lower bound (ELBO) with respect to each parameter and carry out optimization via stochastic gradient ascent, whereas the cited paper uses a more involved coordinate ascent scheme.

The proposed variational distribution is given by:

$$
\begin{aligned}
\theta_k | a_k, b_k & \sim \text{Beta}(a_k, b_k) \equiv q(\theta_k|a_k,b_k) & [\forall k=1,\ldots,K-1]\\
\mu_k | c_k, d_k & \sim \text{Normal}(c_k, d_k) \equiv q(\mu_k|c_k,d_k) & [\forall k=1,\ldots,K] \\ 
\tau_k | e_k, f_k & \sim \text{Gamma}(e_k, f_k) \equiv q(\tau_k|e_k,f_k)& [\forall k=1,\ldots,K] \\
\pi_k | \theta_1,\ldots,\theta_k & = \theta_k\prod_{\ell=1}^{k-1}(1-\theta_\ell) & [\forall k=1,\ldots,K-1]\\
\pi_K | \pi_1,\ldots,\pi_{K-1} & = 1 - \sum_{k=1}^{K-1}\pi_k \\ 
k_n | \pi_1,\ldots,\pi_K & \sim \text{Categorical}(\pi_1,\ldots,\pi_K) & [\forall n=1,\ldots,N] \\
x_n | k_n, \mu_{k_n}, \tau_{k_n} & \sim \text{Normal}(\mu_{k_n}, \tau^{-1}_{k_n}) & [\forall n=1,\ldots,N]
\end{aligned}
$$

Hence, the ELBO is:

$$
\begin{aligned}
\text{ELBO} = \mathbb{E}_q & \left[\sum_{n=1}^N\log\left(\sum_{k=1}^K\pi_k p(x_n|\mu_k,\tau_k)\right)\right] - \sum_{k=1}^{K-1}\text{KL}[q(\theta_k|a_k, b_k)\|p(\theta)] - \sum_{k=1}^{K}\text{KL}[q(\mu_k|c_k, d_k)\|p(\mu)] - \sum_{k=1}^{K}\text{KL}[q(\tau_k|e_k, f_k)\|p(\tau)].
\end{aligned}
$$

We approximate the expectation above with Monte Carlo integration and gradients follow directly via application of the reparameterization trick. As a practical matter, proper initialization of the variational parameters is important since the problem is nonconvex. In the self-contained example, we set $a_k=1$ and $b_k=100$ so that gradients can flow to each cluster at the early stages of training; all other variational parameters are initialized to values drawn from a mean zero normal distribution with small variance. Another thing worth noting is that many of the variational parameters are required to be constrained -- for instance, parameters of the gamma distribution must be positive. These constraints are addressed via reparameterization, e.g., if a variable is restricted to the positive reals, we use an invertible mapping $h: \mathbb{R}\to\mathbb{R}_+$ (e.g., the softplus function) such that optimization is carried out over an unconstrained variable which, when passed through $h$, satisfies the required constraint.